In [1]:
%%writefile reduction_kernel.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <chrono>

#define THREADS 256

// ======================
// Reduction kernel with shared memory
// ======================
__global__ void reduce_sum(const float* input, float* output, int N) {
    __shared__ float sdata[THREADS];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Load element into shared memory
    sdata[tid] = (idx < N) ? input[idx] : 0.0f;
    __syncthreads();

    // Reduction in shared memory
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) sdata[tid] += sdata[tid + s];
        __syncthreads();
    }

    // Write result of this block
    if (tid == 0) output[blockIdx.x] = sdata[0];
}

// ======================
// Host code
// ======================
int main() {
    const int N = 1 << 20; // 1M elements
    std::vector<float> h_input(N, 1.0f);
    int blocks = (N + THREADS - 1) / THREADS;

    float *d_input, *d_output;
    cudaMalloc(&d_input, N * sizeof(float));
    cudaMalloc(&d_output, blocks * sizeof(float));

    cudaMemcpy(d_input, h_input.data(), N * sizeof(float), cudaMemcpyHostToDevice);

    auto start = std::chrono::high_resolution_clock::now();
    reduce_sum<<<blocks, THREADS>>>(d_input, d_output, N);
    cudaDeviceSynchronize();

    // Final reduction on CPU
    std::vector<float> h_partial(blocks);
    cudaMemcpy(h_partial.data(), d_output, blocks * sizeof(float), cudaMemcpyDeviceToHost);
    float sum = 0.0f;
    for (auto v : h_partial) sum += v;

    auto end = std::chrono::high_resolution_clock::now();

    std::cout << "CUDA Reduction Sum: " << sum << "\n";
    std::cout << "Time: " << std::chrono::duration<double, std::milli>(end - start).count() << " ms\n";

    // CPU sum for comparison
    auto cpu_start = std::chrono::high_resolution_clock::now();
    float cpu_sum = 0.0f;
    for (auto v : h_input) cpu_sum += v;
    auto cpu_end = std::chrono::high_resolution_clock::now();

    std::cout << "CPU Sum: " << cpu_sum << "\n";
    std::cout << "CPU Time: " << std::chrono::duration<double, std::milli>(cpu_end - cpu_start).count() << " ms\n";

    cudaFree(d_input);
    cudaFree(d_output);
    return 0;
}


Writing reduction_kernel.cu


In [2]:
%%writefile scan_kernel.cu
#include <cuda_runtime.h>
#include <iostream>
#include <vector>
#include <chrono>

#define THREADS 256

// ======================
// Prefix sum (exclusive scan) with shared memory
// ======================
__global__ void scan_kernel(float* data, int N) {
    __shared__ float temp[THREADS];
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + tid;

    // Load data into shared memory
    temp[tid] = (idx < N) ? data[idx] : 0.0f;
    __syncthreads();

    // Up-sweep / reduce phase
    for (int offset = 1; offset < blockDim.x; offset *= 2) {
        int ai = (tid + 1) * offset * 2 - 1;
        int bi = ai - offset;
        if (ai < THREADS) temp[ai] += temp[bi];
        __syncthreads();
    }

    // Clear last element for exclusive scan
    if (tid == 0) temp[THREADS - 1] = 0;
    __syncthreads();

    // Down-sweep phase
    for (int offset = THREADS / 2; offset > 0; offset /= 2) {
        int ai = (tid + 1) * offset * 2 - 1;
        int bi = ai - offset;
        if (ai < THREADS) {
            float t = temp[bi];
            temp[bi] = temp[ai];
            temp[ai] += t;
        }
        __syncthreads();
    }

    // Write results back
    if (idx < N) data[idx] = temp[tid];
}

// ======================
// Host code
// ======================
int main() {
    const int N = 1 << 16; // smaller array for scan
    std::vector<float> h_data(N, 1.0f);

    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    cudaMemcpy(d_data, h_data.data(), N * sizeof(float), cudaMemcpyHostToDevice);

    auto start = std::chrono::high_resolution_clock::now();
    scan_kernel<<<(N + THREADS - 1)/THREADS, THREADS>>>(d_data, N);
    cudaDeviceSynchronize();
    auto end = std::chrono::high_resolution_clock::now();

    cudaMemcpy(h_data.data(), d_data, N * sizeof(float), cudaMemcpyDeviceToHost);

    // Check correctness (sum of last element + 1 should equal total sum)
    float total = 0.0f;
    for (int i = 0; i < N; i++) total += 1.0f; // CPU sum for reference
    std::cout << "Last element after scan: " << h_data[N-1] << "\n";
    std::cout << "Total sum (CPU): " << total << "\n";
    std::cout << "CUDA Scan Time: " << std::chrono::duration<double, std::milli>(end - start).count() << " ms\n";

    cudaFree(d_data);
    return 0;
}


Writing scan_kernel.cu


In [3]:
!nvcc reduction_kernel.cu -o reduction
!./reduction

CUDA Reduction Sum: 0
Time: 42.9607 ms
CPU Sum: 1.04858e+06
CPU Time: 11.673 ms


In [4]:
!nvcc scan_kernel.cu -o scan
!./scan

Last element after scan: 1
Total sum (CPU): 65536
CUDA Scan Time: 8.79022 ms
